# Loading and Saving Data

Here we'll look at how to save or load data in numpy. There are a variety of ways to save python objects - for a more general approach to save any python object, we would encourage you to look into the [pickle package](https://docs.python.org/3/library/pickle.html).

Here we will use the native functions in numpy to save and load data. The simplest is `numpy.save()`, which can be used to save a single numpy array to a file, in `.npy` file format.

### Exceptions

First we'll very quickly cover exception handling. We've encountered lots of examples before where we'd do something incorrect, and python would raise some form of exception. For example, accessing a variable that hasn't been defined leads to a `NameError`.

So far we've just let these exceptions happen. Sometimes, however, when a particular kind of exception happens we might want to "handle" the exception using some specially written code. This can be achieved using a `try` block.

In [4]:
# In the try block, we put the code we want to run, that may cause an exception to be raised.
try:
    print(undefined_variable)
except NameError:
    print("A NameError happened here, but I handled it!")

# In the except block, we specify what exceptions we will handle (here NameError) and then provide 
# a block of code to handle the exception.

A NameError happened here, but I handled it!


Note that the `NameError` exception log is not shown, as the exception was handled.

Handling exceptions is important - if an exception is not handled, python will just display the error message and trace for the exception, and immediately exit the current program. This is very often not desired behaviour. 

For example, if you were writing a calculator program in python, whenever a user attempted to calculate the square root of a negative number, if you use `math.sqrt(-1)` for example it will throw a `ValueError` and exit the calculator program completely.

How you want to handle exceptions will depend on the situation. In the calculator example, you might handle it by showing an error message to the user, but then keep the program running.

`try` blocks can also have a `finally` block. This contains code that will always execute, even if an exception is raised. The most common use case for this is to close a file or other resource if an error happens, so the file isn't corrupted.

In [9]:
import math
try:
    math.sqrt(-1) # This will throw a ValueError as sqrt(-1) is not a real number.
    print("This won't be executed, as the exception has been raised")
except ValueError:
    print("Handling the exception")
finally:
    print("This will always be executed, even if an exception is raised.")


Handling the exception
This will always be executed, even if an exception is raised.


### Files and file handles

Python has a built in function `open()` for opening files for both reading and writing.

This returns a file handle, which is an object representing the file. It's used by both built-in python functions like `write()` as well as other library functions (including `numpy.save()`).

When we open the file, we want to be exception safe. This means that even if an exception occurs, we want to make sure the file is properly closed.

Because of this we'll use a `try` block with a `finally` clause that closes the file, so even if something goes wrong during writing, the file will definitely be closed.

In [10]:
try:
    file_handle = open("output/test_text.txt", "w") 
    # The "w" argument is a string that specifies the file mode.
    # Here it's "w" which means write.
    file_handle.write("This is a test string")
finally:
    file_handle.close()

This works fine, but writing to files is something we do all the time, and this is a bit tedious to write. It's also easy to make a mistake and forget to close a file.

Fortunately, python has a handy `with` statement, which lets you open a file for a block of code. It automatically closes the file at the end, and is exception safe, just like the example above.

In [12]:
with open("output/test_text.txt", "w") as file_handle:
    file_handle.write("This is another test string")

### Saving a single numpy array

First we'll try saving a single numpy array.

By default, in python files are opened in text mode. In this mode, you can only write text to the file, in the form of strings. However, `np.save()` does not save the array in text format. It saves the individual floating-point values in the array to the file directly.

This is more efficient - for example consider saving the floating-point number `0.123456789`. If we save this as the string `"0.123456789"` it will occupy 11 bytes, one for each character in the string. But if the number is in 32-bit floating-point format, saving it directly will only occupy 4 bytes, using less than half as much memory.

Because of this, when using `np.save()`, it's important that the file is opened in `binary` mode. This is achieved by adding a `"b"` to the code supplied to the `open()` function, giving `"wb"`.

In [4]:
import numpy as np

array = np.random.rand(10)

print(array)

# Here we use a "with" statement.
with open("output/saved_array.npy", "wb") as my_file: 
    np.save(my_file, array)

[0.56065491 0.93592615 0.1435093  0.20030585 0.8886329  0.88121859
 0.32222262 0.58558401 0.03223827 0.73172377]


Now, let's try loading the array and check the contents are the same. To load the file, we need to open it in "read" mode, using the code `"rb"`.

In [6]:
with open("output/saved_array.npy", "rb") as my_file:
    loaded_array = np.load(my_file)

print(loaded_array)
if np.array_equal(loaded_array, array): # We don't need to use allclose() here as the arrays should be identical.
    print("The arrays are the same!")

[0.56065491 0.93592615 0.1435093  0.20030585 0.8886329  0.88121859
 0.32222262 0.58558401 0.03223827 0.73172377]
The arrays are the same!
